In [ ]:
# Imports
import csv
import pandas as pd


# Map range to hour value
def map_hours(hours):
    match int(hours):
        case 1:
            return 20
        case 2:
            return 25
        case 3:
            return 35
        case 4:
            return 40
        case 5:
            return 50
        case _:
            return None

# Convert busy times to a dictionary that can be analyzed
def map_busy_times(times: list[str]):
    # Initialize dictionary with empty lists for each day
    busy_times = {
        'Montag': [],
        'Dienstag': [],
        'Mittwoch': [],
        'Donnerstag': [],
        'Freitag': [],
        'Samstag': [],
        'Sonntag': []
    }

    # Assuming specific hours are represented by the index positions of times
    # e.g., times[0] corresponds to 8-9, times[1] to 9-10, etc., starting at 8.
    for index, time in enumerate(times):
        # Check if the string is not empty
        if time:
            # Split days
            split_days = time.split(';')
            # Calculate the corresponding hour based on index
            hour = 8 + index
            for day in split_days:
                match day:
                    case 'Montag':
                        busy_times['Montag'].append(hour)
                    case 'Dienstag':
                        busy_times['Dienstag'].append(hour)
                    case 'Mittwoch':
                        busy_times['Mittwoch'].append(hour)
                    case 'Donnerstag':
                        busy_times['Donnerstag'].append(hour)
                    case 'Freitag':
                        busy_times['Freitag'].append(hour)
                    case 'Samstag':
                        busy_times['Samstag'].append(hour)
                    case 'Sonntag':
                        busy_times['Sonntag'].append(hour)

    return busy_times

In [ ]:
# Variables
csv_data = r'path'

# Create an empty DataFrame with the desired column names
column_names = ['Stärken', 'Schwächen', 'Termine', 'Zeit', 'ZeitMap', 'Ideen', 'Motivation', 'Ziel']
data = pd.DataFrame(columns=column_names)
map_data = []

# Read CSV file
with open(csv_data, mode='r') as file:
    csv_reader = csv.reader(file)
    
    # Skip header
    next(csv_reader)
    
    # Load and convert rows
    for row in csv_reader:
        new_row = row[1:4] + [map_hours(row[4]), map_busy_times(row[5:18])] + row[18:]
        map_data.append(new_row)
        row_dict = {column_names[i]: new_row[i] for i in range(len(new_row))}
        data = data._append(row_dict, ignore_index=True)

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors


# Aggregate data
def aggregate_availability(data_schedule_list):
    # Create a dictionary to store aggregated availability counts
    hours = range(8, 21)  # 8 AM to 8 PM
    days_of_week = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']
    availability = pd.DataFrame(0, index=hours, columns=days_of_week)

    for entry in data_schedule_list:
        if len(entry) > 4 and isinstance(entry[4], dict):  # Check if the availability data exists
            for day, hours_list in entry[4].items():
                for hour in hours_list:
                    if day in availability.columns and hour in availability.index:
                        availability.at[hour, day] += 1

    return availability

# Heatmap
def plot_heatmap(availability_matrix):
    # Create custom color colormap
    colors = ["blue", "red"]  # 'blue' for least busy, 'red' for most busy
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_blue_red", colors, N=100)
    
    # Create the heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(availability_matrix, annot=True, cmap=cmap, cbar_kws={'label': 'Anzahl besch. Personen'})
    plt.ylabel("Tageszeit (Stunden)")
    plt.yticks(rotation=0)  # Keep the hour labels horizontal for better readability
    plt.show()

In [ ]:
# Aggregate availability data and plot it as a heatmap
# plot_heatmap(aggregate_availability(map_data))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def plot_text_network(personas, key):
    # Create a graph
    graph_to_plot = nx.Graph()

    # Variable to keep track of the previous node to potentially create edges if needed
    previous_node = None

    # Add nodes and edges based on your data. Here's a simplistic approach:
    for persona in personas[key]:
        current_node = persona  # Extract the relevant information using the key
        graph_to_plot.add_node(current_node)  # Add each item as a node


    # Draw the graph
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(graph_to_plot)  # positions for all nodes
    nx.draw(graph_to_plot, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=3000, font_size=9)
    plt.title('Text Input Visualization')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def insert_line_breaks(text, max_width=35):
    """ Insert line breaks into text based on maximum width allowed. """
    words = text.split()
    lines = []
    current_line = []
    current_length = 0

    for word in words:
        if current_length + len(word) > max_width:
            lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word) + 1  # include space
        else:
            current_line.append(word)
            current_length += len(word) + 1  # include space
    lines.append(' '.join(current_line))  # Add last line
    return '\n'.join(lines)

def plot_texts(texts):
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

    annotations = []
    for text in texts:
        text = insert_line_breaks(text)
        x, y = np.random.rand(2)
        while True:
            test_annotation = ax.text(x, y, text, ha='center', va='center', fontsize=15, 
                                      bbox=dict(facecolor='yellow', alpha=0.5, boxstyle='round,pad=0.5'))
            plt.draw()
            bbox = test_annotation.get_window_extent()
            if not any(bbox.overlaps(other.get_window_extent()) for other in annotations):
                annotations.append(test_annotation)
                break
            test_annotation.remove()  # Remove the annotation if it overlaps any others
            x, y = np.random.rand(2)  # Pick a new random position and try again

    plt.show()

In [ ]:
# Plot time data
plot_heatmap(aggregate_availability(map_data))
plot_texts(data['Termine'])

# Make scatter with Zeit and tags

In [ ]:
# Plot the text data
print('Worin siehst du deine Stärken und wie können diese das Projekt positiv beeinflussen?\n')
plot_texts(data['Stärken'])

In [ ]:
print('Was liegt dir weniger?')
plot_texts(data['Schwächen'])

In [ ]:
print('Welche Ideen / Einfälle sind dir während des Einarbeitens in das Projekt gekommen?')
plot_texts(data['Ideen'])

In [ ]:
print('Was ist deine Motivation im Projekt?')
plot_texts(data['Motivation'])

In [ ]:
print('Was ist dein angestrebtes Ziel?')
plot_texts(data['Ziel'])